---
## Projectr : 하루시작 지하철 혼잡도 분석 Ver. 02
## Description : 
    - 서울 교통공사 역별 일별 시간대별 승하차인원 정보
    [source link] (https://data.seoul.go.kr/dataList/OA-12921/F/1/datasetView.do#)
    
## Author : Forrest Dpark (분석 담당)
## Date : 2024.05.31 ~
## Detail : 
## Update: 
    * 2024.06.02 pdg : 일별 데이터 정제 
        - 기존 테스트 데이터 의 일별 데이터가 없는 부분 참고하여 일별 데이터 추출 및 데이터 칼럼 정제를 시작.
    * 2024.06.04 pdg : ppt 만들며 정리 
        - 데이터 정제중 일부 프린트 결과 명확하게 수정함. 
        - 마지막 테이블에 환승역수 관련 칼럼을 추가함. 
    * 2024.06.09 by pdg : 군자, 상봉 검색 오류 이슈 해결 
        - 모듈 대체 할수잇는 코드 대체 
        - 문제파악: station_info 를 만들때 역명 별로 역사코드를 하나씩만 지정되어서 환승역 은 저장이 안됬다. 
        따라서 군자역은 5호선 코드만 저장되고 7호선 코드는 지정되지 않아 나중에 테이블 머지할때 
        사라지게 된것!! 
        또한 상봉역 같은 경우 () 표시가 있었기에 최종테이블 검색 조건이 == 이어서 사라지게된것!!
        - duplicate drop 사용하여 해결. 해결된 함수 모듈화 완료
    * 2024.06.10 by pdg : 08년도 부터 24년까지 데이터 추가 
        - 관찰결과 엑셀데이터인 17,18,19 년도 자료는 칼럼이름이 unnamed 로 나와서 수정해야함. 
        - 모든데이터 받는데 1분 30 초 걸림. 
    * 2024.60.12 by pdg : 데이터 년도 확장, 학습테이블 호선 데이터 확장. 
        - dictionary 는 git 에 올리면 안됨 . 512 Mb 분량이어서 깃에서 에러가 터지면 다지워야함... git에 영향을 받지안토록 따로 보관하고 엑셀파일은 다 지우도록 하자.. 
        - 호선을 확장하는 작업도 해야함.
        - 모든역을 학습한 모델을 만들어도 좋지 않을까? -> 모든역에대한 예측을 하는 모델을 만들것. 


In [2]:
import pandas as pd ,numpy as  np # Data manipulatioin
import seaborn as sns,matplotlib.pyplot as plt # visiulization
import warnings;warnings.filterwarnings('ignore') # 경고 무시 
# 기본 세팅
from Functions import Service
Service.plotSetting("seaborn-v0_8")

___## OS platform 한글 세팅완료 ## ___


In [ ]:
# Data loading 
subway_dict= {
'subway23_0' : pd.read_csv("../Data/교통공사데이터/서울교통공사_역별 일별 시간대별 승하차인원_20230630.csv", encoding='euc-kr'),
'subway23_1' : pd.read_csv("../Data/교통공사데이터/서울교통공사_역별 일별 시간대별 승하차인원 정보_20231031.csv", encoding='euc-kr'),
'subway23_2' : pd.read_csv("../Data/교통공사데이터/서울교통공사_역별 일별 시간대별 승하차인원 정보_23.11_24.01.csv", encoding='euc-kr'),
'subway22' : pd.read_csv("../Data/교통공사데이터/서울교통공사_역별 일별 시간대별 승하차인원 정보_20221231.csv", encoding='euc-kr'),
"subway21" : pd.read_csv("../Data/교통공사데이터/서울교통공사 2021년 일별 역별 시간대별 승하차인원(1_8호선).csv", encoding='euc-kr'),
'subway20' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2020년 일별 역별 시간대별 승하차인원(1_8호선).csv", encoding='euc-kr'),
'subway19' : pd.read_excel("../Data/교통공사데이터/서울교통공사 2019년 일별 역별 시간대별 승하차인원(1_8호선).xlsx",header=None),
'subway18' : pd.read_excel("../Data/교통공사데이터/서울교통공사 2018년 일별 역별 시간대별 승하차인원(1_8호선).xlsx",header=None),
'subway17' : pd.read_excel("../Data/교통공사데이터/서울교통공사 2017년 일별 역별 시간대별 승하차인원(1_8호선).xlsx",header=None),
'subway16' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2016년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr'),
'subway15' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2015년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr'),
'subway14' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2014년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr'),
'subway13' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2013년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr'),
'subway12' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2012년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr'),
'subway11' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2011년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr'),
'subway10' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2010년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr'),
'subway09' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2009년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr'),
'subway08' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2008년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr')
}


In [ ]:

def dataInfoPrint(data):
    Service.dataInfoProcessing(data)
    return data
for key in subway_dict.keys():
    subway_dict[key]=dataInfoPrint(subway_dict[key])
    print(f"{key.split('subway')[-1]}년도 데이터 ")
    subway_dict[key].head(3)

In [3]:
pd.read_excel("../Data/교통공사데이터/서울교통공사 2017년 일별 역별 시간대별 승하차인원(1_8호선).xlsx",header=None)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(단위:명)
1,날짜,구분,호선,역번호,역명,구분,05~06,06~07,07~08,08~09,...,16~17,17~18,18~19,19~20,20~21,21~22,22~23,23~24,00 ~ 01,합 계
2,2017-01-01,신정,1,150,서울역,승차,470,286,397,786,...,3610,3858,3376,3384,2537,3914,2660,917,112,43267
3,2017-01-01,신정,1,150,서울역,하차,278,880,859,964,...,2743,3074,2492,2120,1798,1618,962,551,179,32931
4,2017-01-01,신정,1,151,시청,승차,204,105,112,162,...,921,955,1043,885,805,730,556,188,5,10443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200747,2017-12-31 00:00:00,휴,8,2825,신흥,하차,9,49,38,53,...,301,326,370,345,205,236,219,165,235,3932
200748,2017-12-31 00:00:00,휴,8,2826,수진,승차,42,76,130,216,...,287,393,288,195,203,147,154,67,116,4396
200749,2017-12-31 00:00:00,휴,8,2826,수진,하차,8,68,76,88,...,235,245,269,305,205,264,217,134,200,4062
200750,2017-12-31 00:00:00,휴,8,2827,모란,승차,29,36,74,87,...,254,204,245,182,167,192,179,138,26,3304
